In [2]:
!conda install -q pytorch torchvision cudatoolkit=11 -c pytorch-nightly

import pandas as pd
import numpy as np 
from torch import nn
import torch
from torchtext import data
from torch.nn  import functional as F
import torch.optim as  optim 

if torch.cuda.is_available():  
  dev = "cuda:0" 

  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)

cipy mkl_fft yarl decorator zlib libsodium oauthlib protobuf freetype blinker cython pillow zeromq olefile prompt-toolkit setuptools cffi vc vs2015_runtime torchvision pytorch numba astunparse urllib3 aiohttp sqlite wincertstore markdown cachetools gast brotlipy tensorboard-plugin-wit numpy-base jpeg libprotobuf pyjwt tensorflow jupyter_client mkl_random typing-extensions pyzmq tk google-auth importlib-metadata jupyter_core numpy ninja pyasn1 jedi pyasn1-modules traitlets werkzeug libuv six opt_einsum colorama pip rsa cryptography ipykernel h5py xz wrapt multidict libtiff cudatoolkit mkl-service python-dateutil async-timeout requests-oauthlib attrs idna zstd keras-preprocessing grpcio pygments:  10%|█         | 17/167 [00:22<03:45,  1.50s/it]
Examining conflict for pyopenssl pyjwt requests urllib3 cryptography oauthlib:  10%|█         | 17/167 [00:26<03:45,  1.50s/it]                                                                                                                        

In [7]:
df = pd.read_csv("data/ara_eng.txt",delimiter="\t",names=["eng","ar"])
df

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...


In [3]:
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl (13.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-06-02 18:37:35.934691: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll


# Tokenizer

In [3]:
import math
import pandas as pd
import torchtext
import torch
import torch.nn as nn
import random
import re
import spacy
from torchtext import data
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.lang.ar import Arabic
from nltk.translate.bleu_score import sentence_bleu
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch import Tensor

random.seed(0)
df = pd.read_csv("data/ara_eng.txt",delimiter="\t",names=["eng","ar"])


spacy_eng = spacy.load("en_core_web_sm")

ar = Arabic()
ar_Tokenizer = Tokenizer(ar.vocab)

def engTokenizer(text):
 return  [word.text for word in spacy_eng.tokenizer(text)] 

def arTokenizer(sentence):
    return  [word.text for word in 
             ar_Tokenizer(re.sub(r"\s+"," ",re.sub(r"[\.\'\"\n+]"," ",sentence)).strip())]



SRC = data.Field(tokenize=engTokenizer,batch_first=False,init_token="<sos>",eos_token="<eos>")
TARGET = data.Field(tokenize=arTokenizer,batch_first=False,tokenizer_language="ar",init_token="ببدأ",eos_token="نهها")

class DataFrameDataset(data.Dataset):

    def __init__(self, df, src_field, target_field, is_test=False, **kwargs):
        fields = [('eng', src_field), ('ar',target_field)]
        samples = []
        for i, row in df.iterrows():
            eng = row.eng 
            ar = row.ar
            samples.append(data.Example.fromlist([eng, ar], fields))

        super().__init__(samples, fields, **kwargs)
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

        
torchdataset = DataFrameDataset(df,SRC,TARGET)

train_data, valid_data = torchdataset.split(split_ratio=0.8, random_state = random.seed(0))

SRC.build_vocab(train_data,min_freq=2)
TARGET.build_vocab(train_data,min_freq=2)  

print(TARGET.vocab.freqs.most_common(50))  



[('في', 10114), ('من', 8806), ('على', 5213), ('ان', 2562), ('عن', 2299), ('العالمية', 2260), ('الاصوات', 2191), ('الى', 1867), ('لا', 1599), ('هذا', 1572), ('ما', 1385), ('التي', 1384), ('هذه', 1157), ('مع', 1077), ('الذي', 847), ('أن', 824), ('ذلك', 808), ('كان', 799), ('لم', 797), ('او', 797), ('الانترنت', 785), ('توم', 733), ('هل', 715), ('و', 715), ('كل', 671), ('بعد', 668), ('هو', 616), ('قبل', 580), ('تم', 572), ('موقع', 562), ('حول', 552), ('عام', 505), ('العالم', 496), ('حيث', 488), ('كما', 485), ('بين', 463), ('اكثر', 448), ('المدون', 447), ('قد', 445), ('غير', 441), ('خلال', 432), ('أنا', 432), ('إلى', 428), ('يوم', 427), ('هناك', 426), ('كانت', 424), ('بعض', 420), ('ايضا', 419), ('هي', 419), ('اي', 418)]


In [2]:
print(TARGET.vocab.freqs.most_common(20))  


[('في', 10114), ('من', 8806), ('على', 5213), ('ان', 2562), ('عن', 2299), ('العالمية', 2260), ('الاصوات', 2191), ('الى', 1867), ('لا', 1599), ('هذا', 1572), ('ما', 1385), ('التي', 1384), ('هذه', 1157), ('مع', 1077), ('الذي', 847), ('أن', 824), ('ذلك', 808), ('كان', 799), ('لم', 797), ('او', 797)]


In [19]:
train_data, valid_data = torchdataset.split(split_ratio=0.8, random_state = random.seed(SEED))

SRC.build_vocab(train_data,min_freq=2)
TARGET.build_vocab(train_data,min_freq=2)  

print(TARGET.vocab.freqs.most_common(10))  




[('في', 10140), ('من', 8760), ('على', 5248), ('ان', 2553), ('عن', 2310), ('العالمية', 2261), ('الاصوات', 2195), ('الى', 1826), ('هذا', 1619), ('لا', 1595)]


In [21]:
BATCH_SIZE = 64



train_iterator,valid_iterator = data.BucketIterator.splits(
    (train_data,valid_data), 
    batch_size = BATCH_SIZE,
    device = device,
    sort=False,
    sort_within_batch=False,
shuffle=True)

In [22]:
class TranslateTransformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        max_len,
    ):
        super(TranslateTransformer, self).__init__()
        self.srcEmbeddings = nn.Embedding(src_vocab_size,embedding_size)
        self.trgEmbeddings= nn.Embedding(trg_vocab_size,embedding_size)
        self.srcPositionalEmbeddings= nn.Embedding(max_len,embedding_size)
        self.trgPositionalEmbeddings= nn.Embedding(max_len,embedding_size)
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(0.1)
        self.src_pad_idx = src_pad_idx
        self.max_len = max_len
    
    def make_src_mask(self, src):
        src_mask = src.transpose(0,1) == self.src_pad_idx

        return src_mask.to(device)

    def forward(self,x,trg):
        src_seq_length = x.shape[0]
        N = x.shape[1]
        trg_seq_length = trg.shape[0]
        #adding zeros is an easy way
        src_positions = (
            torch.arange(0, src_seq_length)
            .reshape(src_seq_length,1)  + torch.zeros(src_seq_length,N) 
        ).to(device)
        
        trg_positions = (
            torch.arange(0, trg_seq_length)
            .reshape(trg_seq_length,1)  + torch.zeros(trg_seq_length,N) 
        ).to(device)


        srcWords = self.dropout(self.srcEmbeddings(x.long()) +self.srcPositionalEmbeddings(src_positions.long()))
        trgWords = self.dropout(self.trgEmbeddings(trg.long())+self.trgPositionalEmbeddings(trg_positions.long()))
        
        src_padding_mask = self.make_src_mask(x)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(device)
        
        
        out = self.transformer(srcWords,trgWords, src_key_padding_mask=src_padding_mask,tgt_mask=trg_mask )
        out= self.fc_out(out)
        return out

In [ ]:
src_vocab_size  = len(SRC.vocab)
print("Size of english vocabulary:",src_vocab_size)

#No. of unique tokens in label
trg_vocab_size =len(TARGET.vocab)
print("Size of arabic vocabulary:",trg_vocab_size)

num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3

max_len= 227
embedding_size= 256
src_pad_idx =SRC.vocab.stoi["<pad>"]


model = TranslateTransformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    max_len
).to(device)